In [1]:
import copy 
import pygame
import threading
import time

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Visualize the game state

In [2]:


# Define puzzle board dimensions and colors
WINDOW_SIZE = 300
GRID_SIZE = 3
GRID_WIDTH = WINDOW_SIZE // GRID_SIZE
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

current_puzzle = None

def draw_grid(screen):
    for i in range(1, GRID_SIZE):
        pygame.draw.line(screen, BLACK, (i * GRID_WIDTH, 0), (i * GRID_WIDTH, WINDOW_SIZE))
        pygame.draw.line(screen, BLACK, (0, i * GRID_WIDTH), (WINDOW_SIZE, i * GRID_WIDTH))

def draw_puzzle(screen, puzzle):
    font = pygame.font.Font(None, 36)
    for row in range(GRID_SIZE):
        for col in range(GRID_SIZE):
            cell_value = puzzle[row][col]
            if cell_value != 0:
                cell_text = font.render(str(cell_value), True, BLACK)
                cell_rect = cell_text.get_rect(center=(col * GRID_WIDTH + GRID_WIDTH // 2, row * GRID_WIDTH + GRID_WIDTH // 2))
                screen.blit(cell_text, cell_rect)

def visualize_puzzle(puzzle):
    # if a node is sent instead of a state get hold of the state instead
    if isinstance(puzzle, Node):
        puzzle = puzzle.get_state()
        
    global current_puzzle
    current_puzzle = puzzle

def visualize_path(path):
    for puzzle in path:
        visualize_puzzle(puzzle)
        time.sleep(1)

def puzzle_thread():
    pygame.init()
    screen = pygame.display.set_mode((WINDOW_SIZE, WINDOW_SIZE))
    pygame.display.set_caption('8-Puzzle Visualization')

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

        if current_puzzle is not None:
            screen.fill(WHITE)
            draw_grid(screen)
            draw_puzzle(screen, current_puzzle)
            pygame.display.flip()

def start_puzzle_thread():
    thread = threading.Thread(target=puzzle_thread)
    thread.daemon = True
    thread.start()

def stop_puzzle_thread():
    pygame.quit()

start_puzzle_thread()

# Graph implementation

In [3]:
class Node:
    
    GRID_SIZE = 3
    
    def __init__(self, state):
        self.__state = copy.deepcopy(state)
        self.__parent = None
        self.__path = []
        self.__children = []
        self.__generate_children()            
    
    # generates all possible moves
    def __generate_children(self):
        self.__children = []
        for i in range(self.GRID_SIZE):
            for j in range(self.GRID_SIZE):
                if self.__state[i][j] == 0:
                    if i > 0:
                        self.__children.append(self.swap(i, j, i - 1, j))
                    if i < self.GRID_SIZE - 1:
                        self.__children.append(self.swap(i, j, i + 1, j))
                    if j > 0:
                        self.__children.append(self.swap(i, j, i, j - 1))
                    if j < self.GRID_SIZE - 1:
                        self.__children.append(self.swap(i, j, i, j + 1))
                    return self.__children
        
    def __str__(self):
        return str(self.__state)    
        
    def __repr__(self):
        return str(self.__state)
    
    # returns a state one move from the current state
    def swap(self, x1, y1, x2, y2):
        new_state = copy.deepcopy(self.__state)
        new_state[x1][y1], new_state[x2][y2] = new_state[x2][y2], new_state[x1][y1] 
        return new_state
    
    def set_parent(self, parent):
        self.__parent = parent
        self.__path = parent.get_path()
        self.__path.append(parent)
    
    def get_path(self):
        return copy.copy(self.__path)
    
    def get_state(self):
        return copy.deepcopy(self.__state)
    
    def get_children(self):
        return copy.deepcopy(self.__children)
    
    def get_parent(self):
        return self.__parent

In [4]:
puzzle1 = [
    [1, 2, 3],
    [8, 0, 4],
    [7, 6, 5]
]

In [5]:
node = Node(puzzle1)
visualize_puzzle(node)

# BFS

In [6]:
def BFS(node, target):
    queue = [node]
    visited = []
    while queue:
        current_node = queue.pop(0)
        current_state = current_node.get_state()
        visited.append(current_state)
        
        if current_state == target:
            final_path = current_node.get_path()
            final_path.append(current_node)
            return final_path
        
        for child in current_node.get_children():
            if child not in visited:
                child = Node(child)
                child.set_parent(current_node)
                queue.append(child)
    return None

In [7]:
target = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8]
]

puzzle1 = [
    [1, 2, 5],
    [3, 4, 8],
    [6, 7, 0]
]
node = Node(puzzle1)
# visualize_puzzle(node)

path = BFS(node, target)
path

[[[1, 2, 5], [3, 4, 8], [6, 7, 0]],
 [[1, 2, 5], [3, 4, 0], [6, 7, 8]],
 [[1, 2, 0], [3, 4, 5], [6, 7, 8]],
 [[1, 0, 2], [3, 4, 5], [6, 7, 8]],
 [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

In [8]:
visualize_path(path)

# DFS

In [9]:
def DFS(node, target):
    stack = [node]
    visited = []
    while stack:
        current_node = stack.pop()
        current_state = current_node.get_state()
        visited.append(current_state)
        
        if current_state == target:
            final_path = current_node.get_path()
            final_path.append(current_node)
            return final_path
        
        for child in current_node.get_children():
            if child not in visited:
                child = Node(child)
                child.set_parent(current_node)
                stack.append(child)
    return None

In [10]:
target = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8]
]

puzzle1 = [
    [1, 2, 5],
    [6, 3, 4],
    [7, 8, 0]
]
node = Node(puzzle1)
# visualize_puzzle(node)

path = BFS(node, target)
path

[[[1, 2, 5], [6, 3, 4], [7, 8, 0]],
 [[1, 2, 5], [6, 3, 4], [7, 0, 8]],
 [[1, 2, 5], [6, 3, 4], [0, 7, 8]],
 [[1, 2, 5], [0, 3, 4], [6, 7, 8]],
 [[1, 2, 5], [3, 0, 4], [6, 7, 8]],
 [[1, 2, 5], [3, 4, 0], [6, 7, 8]],
 [[1, 2, 0], [3, 4, 5], [6, 7, 8]],
 [[1, 0, 2], [3, 4, 5], [6, 7, 8]],
 [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

In [11]:
visualize_path(path)

# A*

In [12]:
import heapq

In [13]:
def manhatten_estimate_cost(state):
    cost = 0
    for i in range(len(state)):
        for j in range(len(state)):
            value = state[i][j]
            if value != 0:
                target_row, target_col = value // 3, value % 3
                cost += abs(i - target_row) + abs(j - target_col)
    return cost

In [22]:
def eaclidean_estimate_cost(state):
    cost = 0
    for i in range(len(state)):
        for j in range(len(state)):
            value = state[i][j]
            if value != 0:
                target_row, target_col = value // 3, value % 3
                cost += ((i - target_row)**2 + (j - target_col)**2)**0.5
    return cost

In [15]:
def a_star(node, target, estimate_function=manhatten_estimate_cost):
    
    # Q values are tuples (weight, insert_order, node). insert order is used to break ties
    queue = [(0, 0, node)]
    
    # Q inserts counter
    i = 1
    
    visited = []
    while queue:
        
        # pop the node with the lowest weight
        current_node = heapq.heappop(queue)[-1]
        current_state = current_node.get_state()
        visited.append(current_state)
        
        # check if goal state is reached
        if current_state == target:
            final_path = current_node.get_path()
            final_path.append(current_node)
            return final_path, len(visited)
        
        for child in current_node.get_children():
            if child not in visited:
                child = Node(child)
                child.set_parent(current_node)
                
                # calculate weight by adding the manhatten distance to goal and the path length
                h_n = len(child.get_path())
                g_n = manhatten_estimate_cost(child.get_state())
                weight = h_n + g_n
                
                # push to priority queue
                heapq.heappush(queue, (weight, i, child))
                i+=1
    return None

In [28]:
target = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8]
]

puzzle1 = [
    [1, 2, 0],
    [6, 3, 5],
    [7, 8, 4]
]
node = Node(puzzle1)


In [31]:
t = time.time()
path, visited_nodes = a_star(node, target, estimate_function=manhatten_estimate_cost)
t2 = time.time()
time_taken = t2 - t
visualize_path(path)
f'visited nodes: {visited_nodes}, path length: {len(path)}, time taken: {time_taken}' 

'visited nodes: 16, path length: 11, time taken: 0.004082679748535156'

In [33]:
t = time.time()
path, visited_nodes = a_star(node, target, estimate_function=eaclidean_estimate_cost)
t2 = time.time()
time_taken = t2 - t

visualize_path(path)
f'visited nodes: {visited_nodes}, path length: {len(path)}, time taken: {time_taken}' 

'visited nodes: 16, path length: 11, time taken: 0.002479076385498047'